In [19]:
from PIL import Image, ImageEnhance, ImageDraw, ImageFont
from colorsys import hls_to_rgb
from IPython.core import display
from matplotlib import pyplot as plt
import io
from django.core.files.uploadedfile import InMemoryUploadedFile
#from apps.adverts.models import AdChannel, AdFormat, Advert, Customer
#import django

%matplotlib inline
#django.setup()

In [23]:
def random_color():
    """ generate random rgb color as tuple of tree integers between 0-255 """
    hue = random.randint(0,256)/256
    lightness = random.randint(100, 150)/256
    saturation = random.randint(50, 220)/256
    rgb = tuple(int(value*256) for value in hls_to_rgb(hue, lightness, saturation))
    return rgb

def create_watermark(text='xxx', angle=0,  thumb_size = (50,50)):
    font = ImageFont.truetype("/usr/share/fonts/truetype/msttcorefonts/arial.ttf", 50)
    base = Image.new('RGBA', font.getsize(text), (0,0,0,0))
    drawing = ImageDraw.Draw(base)
    drawing.text((0,0), text, font=font, fill='white')
    base = base.rotate(angle=angle, expand=1, resample=Image.BICUBIC)
    base.thumbnail(thumb_size)
    return base

def reduce_opacity(im, opacity):
    """Returns an image with reduced opacity."""
    assert opacity >= 0 and opacity <= 1
    if im.mode != 'RGBA':
        im = im.convert('RGBA')
    else:
        im = im.copy()
    alpha = im.split()[3]
    alpha = ImageEnhance.Brightness(alpha).enhance(opacity)
    im.putalpha(alpha)
    return im

def watermark(im, mark, position, opacity=1):
    """Adds a watermark to an image."""
    if opacity < 1:
        mark = reduce_opacity(mark, opacity)
    if im.mode != 'RGBA':
        im = im.convert('RGBA')
    # create a transparent layer the size of the image and draw the
    # watermark in that layer.
    layer = Image.new('RGBA', im.size, (0,0,0,0))
    if position == 'tile':
        for y in range(0, im.size[1], mark.size[1]):
            for x in range(0, im.size[0], mark.size[0]):
                layer.paste(mark, (x, y))
    elif position == 'scale':
        # scale, but preserve the aspect ratio
        ratio = min(
            float(im.size[0]) / mark.size[0], float(im.size[1]) / mark.size[1])
        w = int(mark.size[0] * ratio)
        h = int(mark.size[1] * ratio)
        mark = mark.resize((w, h))
        layer.paste(mark, ((im.size[0] - w) // 2, (im.size[1] - h) // 2))
    else:
        layer.paste(mark, position)
    # composite the watermark with the layer
    return Image.composite(layer, im, layer)

def dummy_image_advert(width, height, watermarktext, labeltext):
    img = Image.new('RGB', (width, height), color=random_color())
    mark = create_watermark(watermarktext, angle=10, thumb_size = (100,100))
    label = create_watermark(labeltext, thumb_size = (width, height))
    img = watermark(img, mark, 'tile', 0.4)
    img = watermark(img, label, (0,0), 1)
    
    img_io = io.BytesIO()
    img.save(img_io, format='JPEG')
    filename = '{name:_<18}{w}x{h}.jpg'.format(name=labeltext, w=width, h=height).replace(' ', '-').lower()
    #print(filename)
    
    django_file = InMemoryUploadedFile(
        file=img_io, 
        field_name=None, 
        name=filename,
        content_type='image/jpeg',
        size= len(img_io.getvalue()) , 
        charset=None
    )
    return django_file

In [25]:
customer, new = Customer.objects.get_or_create(name='universitas')
delete_us = Advert.objects.filter(description__startswith='dummy')

for d in delete_us:
    d.imagefile.delete()
    d.delete()

for adformat in AdFormat.objects.all():
    new_image = dummy_image_advert(
        adformat.width, adformat.height, 
        watermarktext='reklame', 
        labeltext=adformat.name,
        )
    
    new_ad = Advert(
        customer = customer,
        description = 'dummy jpg {}'.format(adformat),
        alt_text = 'dummy',
        link = 'http://dummy.com',
        imagefile = new_image,
    )
    new_ad.save()
    new_ad.ad_type = new_ad.determine_ad_type()
    new_ad.save()

universitas DEBUG                 models.py:179  (upload_folder) - adverts/universitas/board-300_________980x300.jpg
universitas DEBUG                 models.py:179  (upload_folder) - adverts/universitas/board-150_________980x150.jpg
universitas DEBUG                 models.py:179  (upload_folder) - adverts/universitas/superboard-500____980x500.jpg
universitas DEBUG                 models.py:179  (upload_folder) - adverts/universitas/netboard__________580x500.jpg
universitas DEBUG                 models.py:179  (upload_folder) - adverts/universitas/articleboard______380x500.jpg
universitas DEBUG                 models.py:179  (upload_folder) - adverts/universitas/skyscraper-xl_____360x1000.jpg
universitas DEBUG                 models.py:179  (upload_folder) - adverts/universitas/skyscraper-bred___360x500.jpg
universitas DEBUG                 models.py:179  (upload_folder) - adverts/universitas/skyscraper________180x500.jpg
universitas DEBUG                 models.py:179  (upload_folder